In [539]:
import numpy as np
import pandas as pd
import random
import shlex
import sys

In [540]:
with open("pcv.txt", "r") as arquivo:
    distancias = []
    cidades = arquivo.readlines()
    for i in range(len(cidades)-1):
        distancias.append(cidades[i + 1].strip('\n'" ").split(" ")[1:])
    #for i in range(len(cidades)):
     #   cidades[i] = cidades[i].strip('\n'" ")

print(distancias)

[['0', '1', '2', '1', '1', '2'], ['1', '0', '7', '1', '4', '3'], ['2', '7', '0', '3', '1', '1'], ['1', '1', '3', '0', '8', '1'], ['1', '4', '1', '8', '0', '1'], ['2', '3', '1', '1', '1', '0']]


In [541]:
class Cidades:
    def _init_(self, cidades = []):
        self.cidades = cidades    
    
    def ler_entrada(self):
        with open("pcv.txt", "r") as arquivo:
            arquivo = arquivo.readlines()
            for i in range(len(arquivo)):
                arquivo[i] = arquivo[i].strip('\n'" ")
        return arquivo
        
    def obter_cidades(self):
        cidades = []
        arquivo = self.ler_entrada()
        cidades.append(arquivo[0].split(" "))
        return cidades[0]
    
    def obter_distancias(self):
        distancias = []
        arquivo = self.ler_entrada()
        for i in range(len(arquivo)-1):
            distancias.append(arquivo[i + 1].split(" ")[1:])
        return distancias


In [542]:
c = Cidades()

In [543]:
c.ler_entrada()


['A B C D E F',
 'A 0 1 2 1 1 2',
 'B 1 0 7 1 4 3',
 'C 2 7 0 3 1 1',
 'D 1 1 3 0 8 1',
 'E 1 4 1 8 0 1',
 'F 2 3 1 1 1 0']

In [544]:
cidades= c.obter_cidades()

In [545]:

class Distancia():
    def _init_(self, cidades):
        self.cidades = cidades

    def obter_distancia_entre_duas_cidades(self, cidade_inicial, cidade_final):
       c =  Cidades()
       index_cidade_inicial = self.cidades.index(cidade_inicial)
       index_cidade_final = self.cidades.index(cidade_final)
       distancias = c.obter_distancias()
       distancia =  int(distancias[index_cidade_inicial][index_cidade_final])
       return distancia

In [548]:
class Individuo():
    def __init__(self, cidades):
        self.cidades = cidades
        self.cromossomo = []
        self.probabilidade = 0
        self.distancia_total = 0
        self.probabilidade = 0
        
        listaCidades = []
        listaCidades.extend(self.cidades)
                
        while len(listaCidades) > 0 :
            cidade_index =  random.randrange(len(listaCidades))           
            self.cromossomo.append(listaCidades[cidade_index])
            listaCidades.pop(cidade_index)
            
        print (self.cromossomo) 
        
    def pressao_seletiva(self, n, tamanho_populacao):
        N = tamanho_populacao
        self.probabilidade  =  (N-n)/(N-1) 
        
    def fitness(self):
        soma_distancia = 0
        
        for i in range(len(self.cromossomo)-1):
            d = Distancia()
            d.cidades = self.cidades
            cidade_inicial = self.cromossomo[i]
            cidade_final = self.cromossomo[i+1]
            soma_distancia += d.obter_distancia_entre_duas_cidades(cidade_inicial, cidade_final)
          
        self.distancia_total = soma_distancia

In [549]:
i=Individuo(cidades)

['A', 'B', 'C', 'F', 'D', 'E']


In [573]:
class AlgoritimoGenetico():
    def __init__(self, cidades, tamanho_populacao = 5,  geracao = 2, prob_cruzamento = 0.9, prob_mutacao = 0.9 ):
        self.tamanho_populacao  = tamanho_populacao
        self.geracao = geracao
        self.prob_cruzamento = prob_cruzamento
        self.prob_mutacao = prob_mutacao
        self.cidades = cidades
        self.populacao = []        
        self.tamanho_roleta = 0   
   
    #gera população inicial
    def init_populacao(self, cidades):
        for i in range(self.tamanho_populacao):
            self.populacao.append(Individuo(cidades))   
            
    def calcula_fitness(self):        
        distancia = []
        for individuo in self.populacao:
            individuo.fitness()
                            
    def ordena_populacao(self):
        self.populacao =  sorted(self.populacao, key=lambda populacao:populacao.distancia_total, reverse = False)  
        
         
    def selecao_roda_roleta_com_ordenacao(self):        
        [(self.populacao[i].pressao_seletiva(i+1, self.tamanho_populacao)) for i in range(self.tamanho_populacao)]
        self.tamanho_roleta = sum([self.populacao[i].probabilidade for i in range(self.tamanho_populacao)])
   
    def selecao(self):
        selecionado =[]
        self.selecao_roda_roleta_com_ordenacao()
        for i in range(self.tamanho_populacao):
            ponto_aleatorio = random.uniform(0, self.tamanho_roleta)
            acc = 0
            for individuo in self.populacao:
                
                print(individuo.cromossomo) 
                if ponto_aleatorio <= acc + individuo.distancia_total:
                    selecionado.append(individuo.cromossomo)
                    print("selecionado")
                    print(selecionado)
                    
                    break
                acc +=individuo.fitness
                
        return selecionado
    
    def cruzamento(self, pai1, pai2):
        gene_index =  random.randrange(len(pai1))
        filho1= pai1[-1:gene_index] + pai2[gene_index :]
        filho2= pai2[-1:gene_index] + pai1[gene_index :]
        filhos+=filho1
        filhos+=filho2
        return filho1, filho2
    
    def seleciona_pais(self): 
        random.shuffle(self.populacao)
        filhos = []
        for i in range(0, len(self.populacao), 2):    
            index_pai1, index_pai2 = i, (i+1) % (len(self.populacao))
            if random.uniform(0, 1) > self.prob_cruzamento:
                continue
            filhos[index_pai1], filhos[index_pai2] = self.cruzamento(self.populacao[index_pai1], self.populacao[index_pai2])
                  
        

    # def init_solucao(self, cidades):
    #     self.init_populacao(cidades)
        
    #     for individuo in self.populacao:
    #         individuo.fitness()
        
    #     #ordena população por fitness
    #     self.ordena_populacao()
        
    #     #print resultado
    #     melhorIndividuo = self.populacao[0]
    #     print("G: %s -> Distancia total: %s Cromossomo: %s" % (
    #         melhorIndividuo.geracao,
    #         melhorIndividuo.distancia_total,
    #         melhorIndividuo.cromossomo
    #     ))

    #     for i in range(self.geracao):     
    #         pais = self.seleciona_pais    
    #         filhos =  self.cruzamento(pais)  
            
            
            #mutação
            #seleciona sobreviventes 
            #calcula fitness
            
          
  

In [574]:
ag =  AlgoritimoGenetico(cidades)
ag.init_populacao(cidades)
ag.calcula_fitness()
ag.ordena_populacao()
ag.selecao_roda_roleta_com_ordenacao()

['B', 'C', 'D', 'F', 'E', 'A']
['E', 'F', 'D', 'B', 'A', 'C']
['F', 'E', 'D', 'C', 'B', 'A']
['F', 'D', 'A', 'C', 'B', 'E']
['E', 'C', 'D', 'F', 'A', 'B']


In [575]:
ag.selecao()

{'cidades': ['A', 'B', 'C', 'D', 'E', 'F'], 'cromossomo': ['E', 'F', 'D', 'B', 'A', 'C'], 'probabilidade': 1.0, 'distancia_total': 6}
ponto aleatorio
0.3917225182942638
['E', 'F', 'D', 'B', 'A', 'C']
6
selecionado
[['E', 'F', 'D', 'B', 'A', 'C']]
{'cidades': ['A', 'B', 'C', 'D', 'E', 'F'], 'cromossomo': ['E', 'C', 'D', 'F', 'A', 'B'], 'probabilidade': 0.75, 'distancia_total': 8}
ponto aleatorio
2.30454251004995
['E', 'F', 'D', 'B', 'A', 'C']
6
selecionado
[['E', 'F', 'D', 'B', 'A', 'C'], ['E', 'F', 'D', 'B', 'A', 'C']]
{'cidades': ['A', 'B', 'C', 'D', 'E', 'F'], 'cromossomo': ['B', 'C', 'D', 'F', 'E', 'A'], 'probabilidade': 0.5, 'distancia_total': 13}
ponto aleatorio
0.591437535818185
['E', 'F', 'D', 'B', 'A', 'C']
6
selecionado
[['E', 'F', 'D', 'B', 'A', 'C'], ['E', 'F', 'D', 'B', 'A', 'C'], ['E', 'F', 'D', 'B', 'A', 'C']]
{'cidades': ['A', 'B', 'C', 'D', 'E', 'F'], 'cromossomo': ['F', 'D', 'A', 'C', 'B', 'E'], 'probabilidade': 0.25, 'distancia_total': 15}
ponto aleatorio
0.2582256304

[['E', 'F', 'D', 'B', 'A', 'C'],
 ['E', 'F', 'D', 'B', 'A', 'C'],
 ['E', 'F', 'D', 'B', 'A', 'C'],
 ['E', 'F', 'D', 'B', 'A', 'C'],
 ['E', 'F', 'D', 'B', 'A', 'C']]